# Notebook 2: XGBoost Training & Comparison
Trains XGBoost on each descriptor CSV from Notebook 1.
Compares R2, MAE, RMSE across all scenarios.

**CSV naming convention:**
- `desc_{TYPE}_w{WINDOW}.csv` for individual types per window
- `desc_ALL_w{WINDOW}.csv` for all types combined per window
- `desc_ALL_multiwindow.csv` for all types across all windows

In [ ]:
import numpy as np
import pandas as pd
import os
import glob
import warnings
from sklearn.model_selection import KFold, cross_val_score, LeaveOneOut
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

In [ ]:
# ============================================================
# PATHS
# ============================================================
BASE_DIR = r"C:\Users\AbCMS_Lab\Desktop\Keshav-DDP"
CSV_DIR = os.path.join(BASE_DIR, "Weight-contribution", "contribution-model")

# Find all descriptor CSVs
csv_files = sorted(glob.glob(os.path.join(CSV_DIR, "desc_*.csv")))
print(f"Found {len(csv_files)} CSV files:")
for f in csv_files:
    name = os.path.basename(f)
    df_tmp = pd.read_csv(f)
    feat_cols = [c for c in df_tmp.columns if c not in 
                 ['uid','Formula','alpha_R','heavy1_el','heavy2_el','heavy1_mass',
                  'heavy2_mass','n_elements','window']]
    print(f"  {name}: {len(df_tmp)} rows, {len(feat_cols)} features")

In [ ]:
# ============================================================
# TRAINING FUNCTION
# ============================================================

# Columns to exclude from features
NON_FEATURE_COLS = {'uid', 'Formula', 'alpha_R', 'heavy1_el', 'heavy2_el',
                     'heavy1_mass', 'heavy2_mass', 'n_elements', 'window'}

def train_and_evaluate(csv_path, n_splits=5):
    """
    Train XGBoost on a descriptor CSV.
    Uses K-Fold CV (and LOO if n < 30).
    Returns dict with metrics.
    """
    df = pd.read_csv(csv_path)
    csv_name = os.path.basename(csv_path)
    
    # Identify feature columns (numeric only, not metadata)
    feature_cols = [c for c in df.columns if c not in NON_FEATURE_COLS]
    # Keep only numeric
    feature_cols = [c for c in feature_cols if df[c].dtype in ['float64','float32','int64','int32']]
    
    if len(feature_cols) == 0:
        return {'csv': csv_name, 'error': 'no features'}
    
    X = df[feature_cols].values
    y = df['alpha_R'].values
    n = len(y)
    
    # Drop rows with NaN
    valid = ~(np.isnan(X).any(axis=1) | np.isnan(y))
    X, y = X[valid], y[valid]
    n = len(y)
    
    if n < 10:
        return {'csv': csv_name, 'error': f'too few samples ({n})'}
    
    # XGBoost with modest params for small dataset
    model = XGBRegressor(
        n_estimators=100,
        max_depth=3,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=1.0,
        reg_lambda=1.0,
        random_state=42,
        verbosity=0
    )
    
    results = {
        'csv': csv_name,
        'n_samples': n,
        'n_features': len(feature_cols),
        'features': feature_cols,
    }
    
    # --- K-Fold CV ---
    k = min(n_splits, n)
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    
    y_pred_kf = np.zeros(n)
    for train_idx, test_idx in kf.split(X):
        model.fit(X[train_idx], y[train_idx])
        y_pred_kf[test_idx] = model.predict(X[test_idx])
    
    results['kfold_R2'] = r2_score(y, y_pred_kf)
    results['kfold_MAE'] = mean_absolute_error(y, y_pred_kf)
    results['kfold_RMSE'] = np.sqrt(mean_squared_error(y, y_pred_kf))
    
    # --- LOO CV (if small dataset) ---
    if n <= 120:
        loo = LeaveOneOut()
        y_pred_loo = np.zeros(n)
        for train_idx, test_idx in loo.split(X):
            model.fit(X[train_idx], y[train_idx])
            y_pred_loo[test_idx] = model.predict(X[test_idx])
        
        results['loo_R2'] = r2_score(y, y_pred_loo)
        results['loo_MAE'] = mean_absolute_error(y, y_pred_loo)
        results['loo_RMSE'] = np.sqrt(mean_squared_error(y, y_pred_loo))
    else:
        results['loo_R2'] = np.nan
        results['loo_MAE'] = np.nan
        results['loo_RMSE'] = np.nan
    
    # --- Feature importance (train on full data) ---
    model.fit(X, y)
    results['feature_importance'] = dict(zip(feature_cols, model.feature_importances_))
    results['train_R2'] = r2_score(y, model.predict(X))
    
    return results

In [ ]:
# ============================================================
# RUN ON ALL CSVs
# ============================================================
all_results = []

for csv_path in csv_files:
    name = os.path.basename(csv_path)
    print(f"Training on {name}...", end=" ")
    res = train_and_evaluate(csv_path)
    all_results.append(res)
    
    if 'error' in res:
        print(f"SKIPPED: {res['error']}")
    else:
        loo_val = res.get('loo_R2', None)
        loo_str = f"{loo_val:.3f}" if isinstance(loo_val, float) and not np.isnan(loo_val) else "N/A"
        print(f"KFold R2={res['kfold_R2']:.3f}, LOO R2={loo_str}, "
              f"n={res['n_samples']}, feats={res['n_features']}")

print("\nDone!")


In [ ]:
# ============================================================
# ABLATION: Base p_frac + ONE elemental feature at a time
# ============================================================
# For each enhanced CSV, take the base p_frac columns,
# then add one elemental feature group at a time.

# Define elemental feature groups (each is a 'one addition')
ELEM_GROUPS = {
    'max_Z4':       ['max_Z4'],
    'max_Z':        ['max_Z'],
    'max_mass':     ['max_mass'],
    'X_diff':       ['X_diff'],
    'X_mean':       ['X_mean'],
    'radius_diff':  ['radius_diff'],
    'radius_mean':  ['radius_mean'],
    'WM':           ['WM_VBM', 'WM_CBM'],
    'WZ4':          ['WZ4_VBM', 'WZ4_CBM'],
    'pZ4':          ['pZ4_VBM', 'pZ4_CBM'],
    'Wr':           ['Wr_VBM', 'Wr_CBM'],
    'WX':           ['WX_VBM', 'WX_CBM'],
}

# Find enhanced CSVs
enhanced_csvs = [f for f in csv_files if 'enhanced' in os.path.basename(f)]
print(f"Enhanced CSVs found: {len(enhanced_csvs)}")
for f in enhanced_csvs:
    print(f"  {os.path.basename(f)}")

ablation_results = []

for csv_path in enhanced_csvs:
    csv_name = os.path.basename(csv_path)
    df = pd.read_csv(csv_path)
    
    # Identify base p_frac columns (handle renamed ones like E_pfrac_VBM_w05)
    base_cols = [c for c in df.columns if 'pfrac' in c.lower() or 'E_pfrac' in c]
    base_cols = [c for c in base_cols if df[c].dtype in ['float64','float32','int64','int32']]
    
    if not base_cols:
        print(f"  {csv_name}: No p_frac columns found, skipping")
        continue
    
    y = df['alpha_R'].values
    
    print(f"\n{'='*60}")
    print(f"{csv_name}")
    print(f"Base columns: {base_cols}")
    print(f"{'='*60}")
    
    # --- Baseline: just p_frac ---
    X_base = df[base_cols].values
    valid = ~(np.isnan(X_base).any(axis=1) | np.isnan(y))
    X_b, y_b = X_base[valid], y[valid]
    
    model = XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1,
                         subsample=0.8, colsample_bytree=0.8,
                         reg_alpha=1.0, reg_lambda=1.0, random_state=42, verbosity=0)
    
    y_pred_base = np.zeros(len(y_b))
    loo = LeaveOneOut()
    for tr, te in loo.split(X_b):
        model.fit(X_b[tr], y_b[tr])
        y_pred_base[te] = model.predict(X_b[te])
    
    base_r2 = r2_score(y_b, y_pred_base)
    base_mae = mean_absolute_error(y_b, y_pred_base)
    
    ablation_results.append({
        'csv': csv_name,
        'added': 'BASELINE (p_frac only)',
        'n_features': len(base_cols),
        'features': base_cols,
        'LOO_R2': base_r2,
        'LOO_MAE': base_mae,
        'delta_R2': 0.0,
    })
    print(f"  BASELINE: {len(base_cols)} feat, LOO R2={base_r2:.3f}, MAE={base_mae:.3f}")
    
    # --- Add one group at a time ---
    for grp_name, grp_cols in ELEM_GROUPS.items():
        # Check which columns exist (handle _w05/_w10 suffixes)
        available = []
        for gc in grp_cols:
            # Direct match
            if gc in df.columns:
                available.append(gc)
            else:
                # Try with window suffixes
                for suffix in ['_w05', '_w10']:
                    if gc + suffix in df.columns:
                        available.append(gc + suffix)
        
        if not available:
            continue
        
        # Check all numeric
        available = [c for c in available if df[c].dtype in ['float64','float32','int64','int32']]
        if not available:
            continue
        
        all_cols = base_cols + available
        X_aug = df[all_cols].values
        valid = ~(np.isnan(X_aug).any(axis=1) | np.isnan(y))
        X_a, y_a = X_aug[valid], y[valid]
        
        y_pred_aug = np.zeros(len(y_a))
        for tr, te in loo.split(X_a):
            model.fit(X_a[tr], y_a[tr])
            y_pred_aug[te] = model.predict(X_a[te])
        
        aug_r2 = r2_score(y_a, y_pred_aug)
        aug_mae = mean_absolute_error(y_a, y_pred_aug)
        delta = aug_r2 - base_r2
        
        marker = '+++' if delta > 0.05 else '++' if delta > 0.02 else '+' if delta > 0 else '--' if delta < -0.05 else '-' if delta < 0 else '='
        
        ablation_results.append({
            'csv': csv_name,
            'added': grp_name,
            'n_features': len(all_cols),
            'features': all_cols,
            'LOO_R2': aug_r2,
            'LOO_MAE': aug_mae,
            'delta_R2': delta,
        })
        print(f"  + {grp_name:<15} ({len(available)} col): LOO R2={aug_r2:.3f}, delta={delta:+.3f} {marker}")

print(f"\nTotal ablation runs: {len(ablation_results)}")


In [ ]:
# ============================================================
# ABLATION SUMMARY: RANKED
# ============================================================
df_abl = pd.DataFrame(ablation_results)
df_abl = df_abl.sort_values('LOO_R2', ascending=False)

print('=' * 90)
print('ABLATION RESULTS RANKED BY LOO R2')
print('=' * 90)
print(df_abl[['csv','added','n_features','LOO_R2','LOO_MAE','delta_R2']].to_string(index=False, float_format='%.3f'))

# Save
abl_path = os.path.join(CSV_DIR, 'ablation_results.csv')
df_abl[['csv','added','n_features','LOO_R2','LOO_MAE','delta_R2']].to_csv(abl_path, index=False)
print(f'\nSaved to {abl_path}')

# Bar chart
fig, ax = plt.subplots(figsize=(12, max(4, len(df_abl)*0.35)))
colors = ['green' if d > 0 else 'red' for d in df_abl['delta_R2']]
labels = df_abl['csv'].str.replace('desc_E_enhanced_','').str.replace('.csv','') + ' + ' + df_abl['added']
ax.barh(range(len(df_abl)), df_abl['LOO_R2'], color=colors, alpha=0.7)
ax.set_yticks(range(len(df_abl)))
ax.set_yticklabels(labels, fontsize=7)
ax.axvline(x=0.398, color='blue', ls='--', lw=1.5, label='Baseline E_w05 (0.398)')
ax.set_xlabel('LOO R2')
ax.set_title('Ablation: p_frac + ONE elemental feature', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.savefig(os.path.join(CSV_DIR, 'ablation_chart.png'), dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# ============================================================
# MULTI-ELEMENTAL COMBOS: p_frac + 2 or 3 best elementals
# ============================================================
# From ablation: radius_mean, max_mass, Wr, max_Z/Z4 helped most.
# Try combining them (2 at a time, 3 at a time).

from itertools import combinations

# Top elemental groups to combine
TOP_ELEM = {
    'radius_mean': ['radius_mean'],
    'max_mass':    ['max_mass'],
    'Wr':          ['Wr_VBM', 'Wr_CBM'],
    'max_Z4':      ['max_Z4'],
    'max_Z':       ['max_Z'],
    'X_mean':      ['X_mean'],
    'X_diff':      ['X_diff'],
    'radius_diff': ['radius_diff'],
}

combo_results = []

for csv_path in enhanced_csvs:
    csv_name = os.path.basename(csv_path)
    df = pd.read_csv(csv_path)
    
    base_cols = [c for c in df.columns if 'pfrac' in c.lower() or 'E_pfrac' in c]
    base_cols = [c for c in base_cols if df[c].dtype in ['float64','float32','int64','int32']]
    if not base_cols:
        continue
    
    y = df['alpha_R'].values
    
    print(f"\n{'='*60}")
    print(f"COMBOS: {csv_name}")
    print(f"{'='*60}")
    
    # Try 2-element combos
    for combo_size in [2, 3]:
        for combo in combinations(TOP_ELEM.keys(), combo_size):
            # Gather columns
            extra_cols = []
            for grp_name in combo:
                for gc in TOP_ELEM[grp_name]:
                    if gc in df.columns:
                        extra_cols.append(gc)
                    else:
                        for suffix in ['_w05', '_w10']:
                            if gc + suffix in df.columns:
                                extra_cols.append(gc + suffix)
            
            extra_cols = [c for c in extra_cols if df[c].dtype in ['float64','float32','int64','int32']]
            if not extra_cols:
                continue
            
            all_cols = base_cols + extra_cols
            # Remove duplicates while preserving order
            seen = set()
            all_cols = [c for c in all_cols if not (c in seen or seen.add(c))]
            
            X = df[all_cols].values
            valid = ~(np.isnan(X).any(axis=1) | np.isnan(y))
            X_v, y_v = X[valid], y[valid]
            
            model = XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1,
                                 subsample=0.8, colsample_bytree=0.8,
                                 reg_alpha=1.0, reg_lambda=1.0, random_state=42, verbosity=0)
            
            # LOO
            y_pred = np.zeros(len(y_v))
            loo = LeaveOneOut()
            for tr, te in loo.split(X_v):
                model.fit(X_v[tr], y_v[tr])
                y_pred[te] = model.predict(X_v[te])
            loo_r2 = r2_score(y_v, y_pred)
            loo_mae = mean_absolute_error(y_v, y_pred)
            
            # KFold
            kf = KFold(n_splits=5, shuffle=True, random_state=42)
            y_pred_kf = np.zeros(len(y_v))
            for tr, te in kf.split(X_v):
                model.fit(X_v[tr], y_v[tr])
                y_pred_kf[te] = model.predict(X_v[te])
            kf_r2 = r2_score(y_v, y_pred_kf)
            
            combo_name = ' + '.join(combo)
            combo_results.append({
                'csv': csv_name,
                'combo': combo_name,
                'n_features': len(all_cols),
                'features': all_cols,
                'LOO_R2': loo_r2,
                'KFold_R2': kf_r2,
                'LOO_MAE': loo_mae,
            })
            
            if loo_r2 > 0.48:
                print(f"  + {combo_name:<40} {len(all_cols)} feat  LOO={loo_r2:.3f}  KF={kf_r2:.3f}")

print(f"\nTotal combo runs: {len(combo_results)}")


In [ ]:
# ============================================================
# COMBO RESULTS: RANKED + SAVE BEST CSVs
# ============================================================
df_combo = pd.DataFrame(combo_results)
df_combo = df_combo.sort_values('LOO_R2', ascending=False)

print('=' * 100)
print('TOP 20 MULTI-ELEMENTAL COMBO RESULTS')
print('=' * 100)
print(df_combo[['csv','combo','n_features','LOO_R2','KFold_R2','LOO_MAE']].head(20).to_string(index=False, float_format='%.3f'))

# Save combo results
df_combo[['csv','combo','n_features','LOO_R2','KFold_R2','LOO_MAE']].to_csv(
    os.path.join(CSV_DIR, 'combo_results.csv'), index=False)

# Save CSVs for top 5 combos that don't already have CSVs
print(f"\n{'='*60}")
print('SAVING TOP 5 COMBO CSVs')
print(f"{'='*60}")

for idx in range(min(5, len(df_combo))):
    row = df_combo.iloc[idx]
    src_csv = os.path.join(CSV_DIR, row['csv'])
    df_src = pd.read_csv(src_csv)
    
    feat_cols = row['features']
    feat_cols = [c for c in feat_cols if c in df_src.columns]
    
    out_cols = ['uid', 'Formula', 'alpha_R'] + feat_cols
    combo_tag = row['combo'].replace(' + ', '_').replace(' ', '')
    base_tag = row['csv'].replace('desc_E_enhanced_','').replace('.csv','')
    csv_name = f"best_combo_{idx+1}_{base_tag}_{combo_tag}.csv"
    csv_path = os.path.join(CSV_DIR, csv_name)
    df_src[out_cols].to_csv(csv_path, index=False)
    print(f"  #{idx+1}: {csv_name}")
    print(f"       LOO R2={row['LOO_R2']:.3f}, KFold R2={row['KFold_R2']:.3f}, {len(feat_cols)} features")
    print(f"       Features: {feat_cols}")


In [ ]:
# ============================================================
# FINAL: TOP 5 OVERALL (ablation + combos + gradient)
# ============================================================
# Merge all results
all_ranked = []

# From ablation
for r in ablation_results:
    all_ranked.append({
        'source': 'ablation',
        'description': r['csv'].replace('desc_E_enhanced_','').replace('.csv','') + ' + ' + r['added'],
        'n_features': r['n_features'],
        'features': r['features'],
        'LOO_R2': r['LOO_R2'],
        'LOO_MAE': r['LOO_MAE'],
        'csv_source': r['csv'],
    })

# From combos
for _, r in df_combo.iterrows():
    all_ranked.append({
        'source': 'combo',
        'description': r['csv'].replace('desc_E_enhanced_','').replace('.csv','') + ' + ' + r['combo'],
        'n_features': r['n_features'],
        'features': r['features'],
        'LOO_R2': r['LOO_R2'],
        'LOO_MAE': r['LOO_MAE'],
        'csv_source': r['csv'],
    })

# From original scenarios (non-ablation)
for r in all_results:
    if 'error' in r:
        continue
    all_ranked.append({
        'source': 'original',
        'description': r['csv'].replace('.csv',''),
        'n_features': r['n_features'],
        'features': r.get('features', []),
        'LOO_R2': r.get('loo_R2', np.nan),
        'LOO_MAE': r.get('loo_MAE', np.nan),
        'csv_source': r['csv'],
    })

df_all = pd.DataFrame(all_ranked)
df_all = df_all.sort_values('LOO_R2', ascending=False)

print('=' * 100)
print('\U0001f3c6 TOP 10 OVERALL MODELS \U0001f3c6')
print('=' * 100)
for idx in range(min(10, len(df_all))):
    r = df_all.iloc[idx]
    print(f"\n#{idx+1}: {r['description']}")
    print(f"    Source: {r['source']}, Features: {r['n_features']}, LOO R2: {r['LOO_R2']:.3f}, MAE: {r['LOO_MAE']:.3f}")
    if isinstance(r['features'], list):
        print(f"    Cols: {r['features']}")

print(f"\n{'='*100}")
print('RECOMMENDATION: Take top 5 and run detailed model comparison (LinReg, RF, XGB, TabNet, etc.)')
print('CSVs for top 5 combos saved as best_combo_*.csv')
print(f"{'='*100}")


In [ ]:
# ============================================================
# RESULTS SUMMARY TABLE
# ============================================================
summary_rows = []
for res in all_results:
    if 'error' in res:
        continue
    summary_rows.append({
        'CSV': res['csv'],
        'n': res['n_samples'],
        'features': res['n_features'],
        'train_R2': res['train_R2'],
        'KFold_R2': res['kfold_R2'],
        'KFold_MAE': res['kfold_MAE'],
        'KFold_RMSE': res['kfold_RMSE'],
        'LOO_R2': res.get('loo_R2', np.nan),
        'LOO_MAE': res.get('loo_MAE', np.nan),
    })

df_summary = pd.DataFrame(summary_rows)
df_summary = df_summary.sort_values('KFold_R2', ascending=False)

print("=" * 100)
print("RESULTS RANKED BY KFold R2")
print("=" * 100)
print(df_summary.to_string(index=False, float_format='%.3f'))

# Save
summary_path = os.path.join(CSV_DIR, "results_summary.csv")
df_summary.to_csv(summary_path, index=False)
print(f"\nSaved to {summary_path}")

In [ ]:
# ============================================================
# VISUALIZATION: R2 COMPARISON BAR CHART
# ============================================================
fig, ax = plt.subplots(figsize=(14, 6))

df_plot = df_summary.sort_values('KFold_R2', ascending=True)
y_pos = range(len(df_plot))

bars = ax.barh(y_pos, df_plot['KFold_R2'], color='steelblue', alpha=0.8, label='KFold R2')
if 'LOO_R2' in df_plot.columns:
    ax.barh(y_pos, df_plot['LOO_R2'], color='coral', alpha=0.5, height=0.4, label='LOO R2')

ax.set_yticks(y_pos)
ax.set_yticklabels(df_plot['CSV'].str.replace('desc_', '').str.replace('.csv', ''), fontsize=8)
ax.set_xlabel('R2 Score', fontsize=12)
ax.set_title('XGBoost R2 by Descriptor Scenario', fontsize=14, fontweight='bold')
ax.axvline(x=0.45, color='red', ls='--', lw=1, alpha=0.7, label='Previous best (0.45)')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig(os.path.join(CSV_DIR, 'r2_comparison.png'), dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ============================================================
# FEATURE IMPORTANCE FOR TOP 3 ABLATION MODELS
# ============================================================
df_abl_sorted = pd.DataFrame(ablation_results).sort_values('LOO_R2', ascending=False)

for idx in range(min(3, len(df_abl_sorted))):
    row = df_abl_sorted.iloc[idx]
    csv_path = os.path.join(CSV_DIR, row['csv'])
    feat_cols = row['features']
    
    df_data = pd.read_csv(csv_path)
    feat_cols = [c for c in feat_cols if c in df_data.columns]
    
    X = df_data[feat_cols].values
    y = df_data['alpha_R'].values
    valid = ~(np.isnan(X).any(axis=1) | np.isnan(y))
    X, y = X[valid], y[valid]
    
    model = XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1,
                         subsample=0.8, colsample_bytree=0.8,
                         reg_alpha=1.0, reg_lambda=1.0, random_state=42, verbosity=0)
    model.fit(X, y)
    
    label = row['csv'].replace('desc_E_enhanced_','').replace('.csv','') + ' + ' + row['added']
    print(f"\n{'='*50}")
    print(f"Model #{idx+1}: {label} (LOO R2={row['LOO_R2']:.3f})")
    print(f"{'='*50}")
    fi = dict(zip(feat_cols, model.feature_importances_))
    for feat, imp in sorted(fi.items(), key=lambda x: x[1], reverse=True):
        bar = '|' * int(imp * 40)
        print(f"  {feat:<25} {imp:.3f} {bar}")


In [ ]:
# ============================================================
# PRED vs ACTUAL SCATTER FOR TOP 5 ABLATION MODELS
# ============================================================
# Sort ablation results by LOO_R2
df_abl_sorted = pd.DataFrame(ablation_results).sort_values('LOO_R2', ascending=False)
top_n = min(5, len(df_abl_sorted))

fig, axes = plt.subplots(1, top_n, figsize=(5*top_n, 5))
if top_n == 1:
    axes = [axes]

for idx in range(top_n):
    row = df_abl_sorted.iloc[idx]
    csv_path = os.path.join(CSV_DIR, row['csv'])
    feat_cols = row['features']
    
    df_data = pd.read_csv(csv_path)
    # Check all feature cols exist
    feat_cols = [c for c in feat_cols if c in df_data.columns]
    
    X = df_data[feat_cols].values
    y = df_data['alpha_R'].values
    valid = ~(np.isnan(X).any(axis=1) | np.isnan(y))
    X, y = X[valid], y[valid]
    
    model = XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1,
                         subsample=0.8, colsample_bytree=0.8,
                         reg_alpha=1.0, reg_lambda=1.0, random_state=42, verbosity=0)
    
    y_pred = np.zeros(len(y))
    loo = LeaveOneOut()
    for tr, te in loo.split(X):
        model.fit(X[tr], y[tr])
        y_pred[te] = model.predict(X[te])
    
    ax = axes[idx]
    ax.scatter(y, y_pred, alpha=0.6, edgecolors='black', linewidth=0.5, s=30)
    lims = [min(y.min(), y_pred.min()) - 0.2, max(y.max(), y_pred.max()) + 0.2]
    ax.plot(lims, lims, 'r--', lw=1.5)
    ax.set_xlabel('Actual', fontsize=10)
    ax.set_ylabel('Predicted', fontsize=10)
    
    label = row['csv'].replace('desc_E_enhanced_','').replace('.csv','') + ' + ' + row['added']
    r2 = r2_score(y, y_pred)
    mae = mean_absolute_error(y, y_pred)
    ax.set_title(f'#{idx+1}: {label}\nR2={r2:.3f}, MAE={mae:.3f}', fontsize=9, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)

plt.suptitle('Top 5 Ablation Models: LOO Predicted vs Actual', fontsize=13, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(os.path.join(CSV_DIR, 'top5_pred_vs_actual.png'), dpi=150, bbox_inches='tight')
plt.show()
